# Atrasos em Voos no Brasil — ANAC (VRA)

**Objetivo**: coletar 3 anos consecutivos do VRA (ANAC), tratar/limpar, fazer engenharia de features e analisar atrasos por aeroporto, companhia, período do dia e dia da semana.

> **Como usar**: Execute célula por célula (ou `Run all`). O notebook baixa os dados, processa e gera tabelas/figuras.

# Parâmetros

In [1]:
YEARS = [2022, 2023, 2024]
FILTER_AIRPORTS_ICAO = None
FILTER_AIRLINES = None

ANAC_BASE = 'https://sistemas.anac.gov.br/dadosabertos/Voos%20e%20opera%C3%A7%C3%B5es%20a%C3%A9reas/Voo%20Regular%20Ativo%20%28VRA%29'
MONTH_DIR = {
    1: '01 - Janeiro', 2: '02 - Fevereiro', 3: '03 - Março', 4: '04 - Abril',
    5: '05 - Maio', 6: '06 - Junho', 7: '07 - Julho', 8: '08 - Agosto',
    9: '09 - Setembro', 10: '10 - Outubro', 11: '11 - Novembro', 12: '12 - Dezembro'
}

AIRPORT_CODES_CSV = 'https://raw.githubusercontent.com/datasets/airport-codes/main/data/airport-codes.csv'
OUTPUT_DIR = 'output'


# Funções gerais

In [2]:
import os, io, sys, textwrap, itertools, math, zipfile, warnings, re
from pathlib import Path
from typing import List, Optional, Tuple
import pandas as pd
import numpy as np
import requests
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 120)
Path(OUTPUT_DIR).mkdir(exist_ok=True)

def robust_read_csv_from_bytes(b: bytes):
    for skip in range(0, 20):
        try:
            df = pd.read_csv(
                io.BytesIO(b),
                sep=';',
                encoding='utf-8',
                engine='python',
                on_bad_lines='skip',
                skiprows=skip
            )
            if any('ICAO' in str(c) or 'Aeródromo' in str(c) for c in df.columns):
                return df
        except Exception:
            continue
    raise ValueError("Não consegui detectar cabeçalho correto do CSV.")

def download(url: str, timeout=120) -> bytes:
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    return r.content

def build_anac_month_url(year: int, month: int) -> Tuple[str, str]:
    fname = f'VRA_{year}{month}.csv'
    folder = f'{ANAC_BASE}/{year}/{MONTH_DIR[month]}'
    url = f'{folder}/{fname}'
    return url, fname

def try_get_vra_month(year: int, month: int) -> Optional[pd.DataFrame]:
    url, fname = build_anac_month_url(year, month)
    try:
        b = download(url)
        df = robust_read_csv_from_bytes(b)
        df['_src_year'] = year
        df['_src_month'] = month
        df['_src_url'] = url
        return df
    except Exception as e:
        print(f'[WARN] {year}-{month:02d}: não consegui baixar/ler {url} -> {e}')
        return None

def load_vra_years(years: List[int]) -> pd.DataFrame:
    frames = []
    for y in years:
        print(f'Baixando {y}...')
        for m in range(1,13):
            dfm = try_get_vra_month(y, m)
            if dfm is not None:
                frames.append(dfm)
    if not frames:
        raise RuntimeError('Nenhum mês foi carregado. Verifique sua rede/anos.')
    df = pd.concat(frames, ignore_index=True)
    return df

def load_airport_codes():
    print('Baixando airport-codes...')
    b = download(AIRPORT_CODES_CSV)
    df = pd.read_csv(io.BytesIO(b))

    for col in ['ident','icao_code','gps_code']:
        if col in df.columns:
            df[col] = df[col].astype(str)

    def pick_icao(row):
        for col in ['icao_code','ident','gps_code']:
            if col in row and isinstance(row[col], str) and len(row[col]) == 4:
                return row[col].upper()
        return np.nan

    df['icao'] = df.apply(pick_icao, axis=1)

    if 'iata_code' in df.columns:
        df['iata'] = df['iata_code'].astype(str).str.upper()

    keep = [c for c in ['icao','iata','name','municipality','iso_country'] if c in df.columns]
    return df[keep].drop_duplicates(subset=['icao']).dropna(subset=['icao'])

def find_datetime_column(df: pd.DataFrame, keywords: List[str]) -> Optional[str]:
    # tenta encontrar coluna de data/hora por palavras-chave
    cand = []
    for c in df.columns:
        lc = c.lower()
        if any(k in lc for k in keywords):
            cand.append(c)
    if not cand:
        return None
    # retorna a mais longa (muitas vezes nome mais descritivo)
    return sorted(cand, key=len, reverse=True)[0]

def parse_datetimes_safe(df: pd.DataFrame, cols: List[str]) -> None:
    for c in cols:
        if c in df.columns:
            df[c] = pd.to_datetime(df[c], errors='coerce')

def coalesce(*args):
    for a in args:
        if a is not None:
            return a
    return None


## 1) Download & Load

In [3]:
vra = load_vra_years(YEARS)
print('Shape VRA:', vra.shape)
print(vra.columns.tolist())
vra.head(3)

Baixando 2022...
Baixando 2023...
Baixando 2024...
Shape VRA: (2845173, 15)
['ICAO Empresa Aérea', 'Número Voo', 'Código Autorização (DI)', 'Código Tipo Linha', 'ICAO Aeródromo Origem', 'ICAO Aeródromo Destino', 'Partida Prevista', 'Partida Real', 'Chegada Prevista', 'Chegada Real', 'Situação Voo', 'Código Justificativa', '_src_year', '_src_month', '_src_url']


,ICAO Empresa Aérea,Número Voo,Código Autorização (DI),Código Tipo Linha,ICAO Aeródromo Origem,ICAO Aeródromo Destino,Partida Prevista,Partida Real,Chegada Prevista,Chegada Real,Situação Voo,Código Justificativa,_src_year,_src_month,_src_url
0,ARG,1241,0,I,SBGR,SABE,2022-01-02 11:00:00,2022-01-02 11:15:00,2022-01-02 13:55:00,2022-01-02 13:45:00,REALIZADO,NaN,2022,1,https://sistemas.anac.gov.br/dadosabertos/Voos%20e%20opera%C3%A7%C3%B5es%20a%C3%A9reas/Voo%20Regular%20Ativo%20%28VR...
1,ARG,1241,0,I,SBGR,SABE,2022-01-03 11:00:00,2022-01-03 11:20:00,2022-01-03 13:55:00,2022-01-03 13:55:00,REALIZADO,NaN,2022,1,https://sistemas.anac.gov.br/dadosabertos/Voos%20e%20opera%C3%A7%C3%B5es%20a%C3%A9reas/Voo%20Regular%20Ativo%20%28VR...
2,ARG,1241,0,I,SBGR,SABE,2022-01-04 11:00:00,2022-01-04 11:15:00,2022-01-04 13:55:00,2022-01-04 13:40:00,REALIZADO,NaN,2022,1,https://sistemas.anac.gov.br/dadosabertos/Voos%20e%20opera%C3%A7%C3%B5es%20a%C3%A9reas/Voo%20Regular%20Ativo%20%28VR...


## 2) Padronização de colunas

In [4]:
import pandas as pd
import unicodedata, re

def normalize_colname(col):
    col = str(col)
    col = unicodedata.normalize("NFKD", col).encode("ascii", "ignore").decode("utf-8")
    col = col.lower()
    col = re.sub(r'[^0-9a-z]+', '_', col)
    return col.strip('_')

vra.columns = [normalize_colname(c) for c in vra.columns]
print(vra.columns.tolist())

for c in ['partida_prevista', 'partida_real', 'chegada_prevista', 'chegada_real']:
    if c in vra.columns:
        vra[c] = pd.to_datetime(vra[c], errors='coerce')

vra['delay_min'] = (vra['partida_real'] - vra['partida_prevista']).dt.total_seconds() / 60
vra['is_atraso_15'] = vra['delay_min'] >= 15

vra['ano'] = vra['partida_prevista'].dt.year
vra['mes'] = vra['partida_prevista'].dt.month
vra['dow'] = vra['partida_prevista'].dt.dayofweek
vra['hora'] = vra['partida_prevista'].dt.hour

def bucket_periodo(h):
    if pd.isna(h): return 'desconhecido'
    h = int(h)
    if 5 <= h < 12: return 'manha'
    if 12 <= h < 18: return 'tarde'
    if 18 <= h <= 23: return 'noite'
    return 'madrugada'

vra['periodo_dia'] = vra['hora'].apply(bucket_periodo)
vra = vra[vra['ano'].isin(YEARS)].copy()

print(vra[['partida_prevista','partida_real','delay_min','is_atraso_15']].head())

# mapeamento fixo com os nomes já normalizados
col_icao_origem = 'icao_aerodromo_origem'
col_icao_dest   = 'icao_aerodromo_destino'
col_cia         = 'icao_empresa_aerea'

col_part_prev   = 'partida_prevista'
col_part_real   = 'partida_real'
col_cheg_prev   = 'chegada_prevista'
col_cheg_real   = 'chegada_real'

col_atraso_min  = None  # não existe coluna pronta de atraso

# as vezes existem colunas explícitas de atraso em minutos
col_atraso_min = next((c for c in vra.columns if 'atras' in c and ('min' in c or 'minuto' in c)), None)

guia = {
    'icao_origem_col': col_icao_origem,
    'icao_dest_col': col_icao_dest,
    'cia_col': col_cia,
    'partida_prevista_col': col_part_prev,
    'partida_real_col': col_part_real,
    'chegada_prevista_col': col_cheg_prev,
    'chegada_real_col': col_cheg_real,
    'atraso_min_col': col_atraso_min,
}
print(guia)
vra.head(2)

['icao_empresa_aerea', 'numero_voo', 'codigo_autorizacao_di', 'codigo_tipo_linha', 'icao_aerodromo_origem', 'icao_aerodromo_destino', 'partida_prevista', 'partida_real', 'chegada_prevista', 'chegada_real', 'situacao_voo', 'codigo_justificativa', 'src_year', 'src_month', 'src_url']
     partida_prevista        partida_real  delay_min  is_atraso_15
0 2022-01-02 11:00:00 2022-01-02 11:15:00       15.0          True
1 2022-01-03 11:00:00 2022-01-03 11:20:00       20.0          True
2 2022-01-04 11:00:00 2022-01-04 11:15:00       15.0          True
3 2022-01-05 11:00:00 2022-01-05 11:20:00       20.0          True
4 2022-01-06 11:00:00 2022-01-06 11:25:00       25.0          True
{'icao_origem_col': 'icao_aerodromo_origem', 'icao_dest_col': 'icao_aerodromo_destino', 'cia_col': 'icao_empresa_aerea', 'partida_prevista_col': 'partida_prevista', 'partida_real_col': 'partida_real', 'chegada_prevista_col': 'chegada_prevista', 'chegada_real_col': 'chegada_real', 'atraso_min_col': None}


,icao_empresa_aerea,numero_voo,codigo_autorizacao_di,codigo_tipo_linha,icao_aerodromo_origem,icao_aerodromo_destino,partida_prevista,partida_real,chegada_prevista,chegada_real,...,src_year,src_month,src_url,delay_min,is_atraso_15,ano,mes,dow,hora,periodo_dia
0,ARG,1241,0,I,SBGR,SABE,2022-01-02 11:00:00,2022-01-02 11:15:00,2022-01-02 13:55:00,2022-01-02 13:45:00,...,2022,1,https://sistemas.anac.gov.br/dadosabertos/Voos%20e%20opera%C3%A7%C3%B5es%20a%C3%A9reas/Voo%20Regular%20Ativo%20%28VR...,15.0,True,2022.0,1.0,6.0,11.0,manha
1,ARG,1241,0,I,SBGR,SABE,2022-01-03 11:00:00,2022-01-03 11:20:00,2022-01-03 13:55:00,2022-01-03 13:55:00,...,2022,1,https://sistemas.anac.gov.br/dadosabertos/Voos%20e%20opera%C3%A7%C3%B5es%20a%C3%A9reas/Voo%20Regular%20Ativo%20%28VR...,20.0,True,2022.0,1.0,0.0,11.0,manha


## 3) Engenharia de Features — atraso, período do dia, dia da semana, ano/mês

In [5]:
# converte possíveis colunas de data/hora
parse_datetimes_safe(vra, [c for c in [col_part_prev, col_part_real, col_cheg_prev, col_cheg_real] if c])

# define atraso (minutos)
# preferência: partida_real - partida_prevista; senão chegada_real - chegada_prevista; senão coluna 'atraso' se existir
delay = None
if col_part_prev and col_part_real and np.issubdtype(vra[col_part_prev].dtype, np.datetime64) and np.issubdtype(vra[col_part_real].dtype, np.datetime64):
    delay = (vra[col_part_real] - vra[col_part_prev]).dt.total_seconds() / 60.0
elif col_cheg_prev and col_cheg_real and np.issubdtype(vra[col_cheg_prev].dtype, np.datetime64) and np.issubdtype(vra[col_cheg_real].dtype, np.datetime64):
    delay = (vra[col_cheg_real] - vra[col_cheg_prev]).dt.total_seconds() / 60.0

if delay is None and col_atraso_min and pd.api.types.is_numeric_dtype(vra[col_atraso_min]):
    delay = vra[col_atraso_min].astype(float)

if delay is None:
    # se não deu pra inferir, zera e marca flag (evita quebrar as análises)
    delay = pd.Series(0, index=vra.index, dtype=float)
    print('[WARN] Não foi possível inferir atraso; usando 0 minutos por padrão.')

vra['delay_min'] = delay
vra['is_atraso_15'] = (vra['delay_min'] >= 15)  # atraso "operacional" >= 15min (padrão comum)

# partida prevista/real (o que existir), senão chegada
base_dt = coalesce(col_part_prev, col_part_real, col_cheg_prev, col_cheg_real)
if base_dt is None:
    # fallback em colunas auxiliares (criado no load)
    base_dt = None

if base_dt:
    vra['ano'] = vra[base_dt].dt.year
    vra['mes'] = vra[base_dt].dt.month
    vra['dow'] = vra[base_dt].dt.dayofweek  # 0=Seg, 6=Dom
    vra['hora'] = vra[base_dt].dt.hour
else:
    vra['ano'] = vra['_src_year']
    vra['mes'] = vra['_src_month']
    vra['dow'] = np.nan
    vra['hora'] = np.nan

def bucket_periodo(h):
    if pd.isna(h): return 'desconhecido'
    h = int(h)
    if 5 <= h < 12: return 'manhã'
    if 12 <= h < 18: return 'tarde'
    if 18 <= h <= 23: return 'noite'
    return 'madrugada'

vra['periodo_dia'] = vra['hora'].apply(bucket_periodo)
vra = vra[vra['ano'].isin(YEARS)].copy()

# filtragens opcionais
if FILTER_AIRPORTS_ICAO is not None and guia['icao_origem_col']:
    vra = vra[vra[guia['icao_origem_col']].astype(str).str.upper().isin([x.upper() for x in FILTER_AIRPORTS_ICAO])].copy()

if FILTER_AIRLINES is not None and guia['cia_col']:
    vra = vra[vra[guia['cia_col']].astype(str).str.upper().isin([x.upper() for x in FILTER_AIRLINES])].copy()

print(vra[['delay_min','is_atraso_15','ano','mes','dow','hora','periodo_dia']].head(5))


   delay_min  is_atraso_15   ano  mes  dow  hora periodo_dia
0       15.0          True  2022    1    6    11       manhã
1       20.0          True  2022    1    0    11       manhã
2       15.0          True  2022    1    1    11       manhã
3       20.0          True  2022    1    2    11       manhã
4       25.0          True  2022    1    3    11       manhã


## 4) Enriquecimento com Airport Codes (ICAO/IATA)

In [6]:
air = load_airport_codes()

icao_col = guia['icao_origem_col']
if icao_col is None:
    # tenta destino se origem não existir
    icao_col = guia['icao_dest_col']

if icao_col:
    vra['icao'] = vra[icao_col].astype(str).str.upper()
    vra = vra.merge(air, how='left', left_on='icao', right_on='icao', suffixes=('','_air'))
else:
    print('[WARN] Não encontrei coluna com ICAO do aeroporto; seguirei sem enriquecimento.')

vra.head(3)

Baixando airport-codes...


,icao_empresa_aerea,numero_voo,codigo_autorizacao_di,codigo_tipo_linha,icao_aerodromo_origem,icao_aerodromo_destino,partida_prevista,partida_real,chegada_prevista,chegada_real,...,ano,mes,dow,hora,periodo_dia,icao,iata,name,municipality,iso_country
0,ARG,1241,0,I,SBGR,SABE,2022-01-02 11:00:00,2022-01-02 11:15:00,2022-01-02 13:55:00,2022-01-02 13:45:00,...,2022,1,6,11,manhã,SBGR,GRU,Guarulhos - Governador André Franco Montoro International Airport,São Paulo,BR
1,ARG,1241,0,I,SBGR,SABE,2022-01-03 11:00:00,2022-01-03 11:20:00,2022-01-03 13:55:00,2022-01-03 13:55:00,...,2022,1,0,11,manhã,SBGR,GRU,Guarulhos - Governador André Franco Montoro International Airport,São Paulo,BR
2,ARG,1241,0,I,SBGR,SABE,2022-01-04 11:00:00,2022-01-04 11:15:00,2022-01-04 13:55:00,2022-01-04 13:40:00,...,2022,1,1,11,manhã,SBGR,GRU,Guarulhos - Governador André Franco Montoro International Airport,São Paulo,BR


## 5) Análises solicitadas

In [7]:
def top_aeroportos_atrasos(df):
    grp = df.groupby('icao', dropna=False)['is_atraso_15'].sum().sort_values(ascending=False)
    return grp.reset_index(name='qtd_atrasos')

def evolucao_atrasos_por_aeroporto(df):
    g = df.groupby(['ano','icao'], dropna=False)['is_atraso_15'].sum().reset_index()
    # calcula variação ano-a-ano no total de atrasos por aeroporto
    g = g.sort_values(['icao','ano']).reset_index(drop=True)
    g['var_yoy'] = g.groupby('icao')['is_atraso_15'].diff()
    return g

def atrasos_totais_por_ano(df):
    return df.groupby('ano', dropna=False)['is_atraso_15'].sum().reset_index(name='atrasos')

def dias_da_semana_por_ano(df):
    name = {0:'Seg',1:'Ter',2:'Qua',3:'Qui',4:'Sex',5:'Sáb',6:'Dom'}
    tmp = df.copy()
    if 'dow' in tmp:
        tmp['dow_name'] = tmp['dow'].map(name)
    else:
        tmp['dow_name'] = np.nan
    out = tmp.groupby(['ano','dow_name'])['is_atraso_15'].sum().reset_index(name='atrasos')
    return out.sort_values(['ano','atrasos'], ascending=[True, False])

def periodo_dia_por_ano(df):
    out = df.groupby(['ano','periodo_dia'])['is_atraso_15'].sum().reset_index(name='atrasos')
    return out.sort_values(['ano','atrasos'], ascending=[True, False])

def cia_que_mais_atraza_por_ano(df, cia_col: Optional[str]):
    if cia_col is None or cia_col not in df.columns:
        return pd.DataFrame(columns=['ano','companhia','atrasos'])
    tmp = df.copy()
    tmp['companhia'] = tmp[cia_col].astype(str).str.upper()
    out = tmp.groupby(['ano','companhia'])['is_atraso_15'].sum().reset_index(name='atrasos')
    return out.sort_values(['ano','atrasos'], ascending=[True, False])

top_aero = top_aeroportos_atrasos(vra)
evo = evolucao_atrasos_por_aeroporto(vra)
tot_ano = atrasos_totais_por_ano(vra)
dow_ano = dias_da_semana_por_ano(vra)
per_ano = periodo_dia_por_ano(vra)
cia_ano = cia_que_mais_atraza_por_ano(vra, guia['cia_col'])

top_aero.head(10)

,icao,qtd_atrasos
0,SBGR,85361
1,SBSP,52053
2,SBKP,30260
3,SBCF,23285
4,SBRJ,20119
5,SBBR,20008
6,SBGL,17160
7,SBRF,16215
8,SBSV,11985
9,SBPA,11601


### Exporta tabelas-chave (CSV)

In [8]:
top_aero.to_csv(f'{OUTPUT_DIR}/aeroportos_mais_atrasos.csv', index=False)
evo.to_csv(f'{OUTPUT_DIR}/evolucao_atrasos_por_aeroporto.csv', index=False)
tot_ano.to_csv(f'{OUTPUT_DIR}/atrasos_totais_por_ano.csv', index=False)
dow_ano.to_csv(f'{OUTPUT_DIR}/dias_semana_atrasos_por_ano.csv', index=False)
per_ano.to_csv(f'{OUTPUT_DIR}/periodo_dia_atrasos_por_ano.csv', index=False)
cia_ano.to_csv(f'{OUTPUT_DIR}/companhia_mais_atraza_por_ano.csv', index=False)

# compacta tudo
with zipfile.ZipFile(f'{OUTPUT_DIR}/resultados_vra.zip', 'w', zipfile.ZIP_DEFLATED) as z:
    for f in [
        'aeroportos_mais_atrasos.csv',
        'evolucao_atrasos_por_aeroporto.csv',
        'atrasos_totais_por_ano.csv',
        'dias_semana_atrasos_por_ano.csv',
        'periodo_dia_atrasos_por_ano.csv',
        'companhia_mais_atraza_por_ano.csv'
    ]:
        z.write(f'{OUTPUT_DIR}/{f}', arcname=f)
print('Arquivos gerados em', OUTPUT_DIR)

Arquivos gerados em output


## 6) Gráficos (matplotlib)

In [9]:
# criar pasta de gráficos
GRAPH_DIR = f"{OUTPUT_DIR}/graficos"
Path(GRAPH_DIR).mkdir(parents=True, exist_ok=True)

# 1) Atrasos por ano (tendência)
plt.figure()
plt.plot(tot_ano['ano'], tot_ano['atrasos'], marker='o')
plt.title('Atrasos (>=15 min) por Ano')
plt.xlabel('Ano'); plt.ylabel('Qtd de atrasos')
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{GRAPH_DIR}/atrasos_por_ano.png", dpi=300, bbox_inches='tight')
plt.close()

# 2) Top 10 aeroportos com mais atrasos (geral)
top10 = top_aero.head(10)
plt.figure()
plt.bar(top10['icao'].astype(str), top10['qtd_atrasos'])
plt.title('Top 10 Aeroportos com Mais Atrasos (>=15 min)')
plt.xlabel('Aeroporto (ICAO)'); plt.ylabel('Qtd de atrasos')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(f"{GRAPH_DIR}/top10_aeroportos.png", dpi=300, bbox_inches='tight')
plt.close()

# 3) Dias da semana com mais atrasos (por ano)
for a in sorted(dow_ano['ano'].dropna().unique()):
    dfa = dow_ano[dow_ano['ano']==a]
    plt.figure()
    plt.bar(dfa['dow_name'].astype(str), dfa['atrasos'])
    plt.title(f'Atrasos por Dia da Semana — {a}')
    plt.xlabel('Dia da semana'); plt.ylabel('Qtd de atrasos')
    plt.grid(True, axis='y')
    plt.tight_layout()
    plt.savefig(f"{GRAPH_DIR}/dias_semana_{a}.png", dpi=300, bbox_inches='tight')
    plt.close()

# 4) Período do dia com mais atrasos (por ano)
for a in sorted(per_ano['ano'].dropna().unique()):
    dfa = per_ano[per_ano['ano']==a]
    plt.figure()
    plt.bar(dfa['periodo_dia'].astype(str), dfa['atrasos'])
    plt.title(f'Atrasos por Período do Dia — {a}')
    plt.xlabel('Período do dia'); plt.ylabel('Qtd de atrasos')
    plt.grid(True, axis='y')
    plt.tight_layout()
    plt.savefig(f"{GRAPH_DIR}/periodo_dia_{a}.png", dpi=300, bbox_inches='tight')
    plt.close()

# 5) Companhia que mais atrasa por ano
if not cia_ano.empty:
    for a in sorted(cia_ano['ano'].unique()):
        dfa = cia_ano[cia_ano['ano']==a].head(10)
        plt.figure()
        plt.bar(dfa['companhia'].astype(str), dfa['atrasos'])
        plt.title(f'Companhias com Mais Atrasos — {a}')
        plt.xlabel('Companhia'); plt.ylabel('Qtd de atrasos')
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.savefig(f"{GRAPH_DIR}/companhias_{a}.png", dpi=300, bbox_inches='tight')
        plt.close()
else:
    print('Coluna de companhia aérea não identificada — pulando gráficos de cias.')

# atualiza o zip incluindo os gráficos
with zipfile.ZipFile(f'{OUTPUT_DIR}/resultados_vra.zip', 'w', zipfile.ZIP_DEFLATED) as z:
    # CSVs
    for f in [
        'aeroportos_mais_atrasos.csv',
        'evolucao_atrasos_por_aeroporto.csv',
        'atrasos_totais_por_ano.csv',
        'dias_semana_atrasos_por_ano.csv',
        'periodo_dia_atrasos_por_ano.csv',
        'companhia_mais_atraza_por_ano.csv'
    ]:
        z.write(f'{OUTPUT_DIR}/{f}', arcname=f)

    # PNGs
    for f in os.listdir(GRAPH_DIR):
        z.write(f"{GRAPH_DIR}/{f}", arcname=f"graficos/{f}")

print('Arquivos (CSV + gráficos) gerados em', OUTPUT_DIR)



Arquivos (CSV + gráficos) gerados em output


## 7) Blocos para Storytelling (resumos automáticos)

In [ ]:
from datetime import datetime

# gera insights em texto
def resumo_geral(tot_ano, top_aero, cia_ano):
    linhas = []
    # tendência
    tr = tot_ano.sort_values('ano')
    if len(tr) >= 2:
        delta = tr['atrasos'].iloc[-1] - tr['atrasos'].iloc[0]
        direcao = 'aumentaram' if delta > 0 else ('diminuíram' if delta < 0 else 'ficaram estáveis')
        linhas.append(f"No período analisado ({int(tr['ano'].min())}-{int(tr['ano'].max())}), os atrasos {direcao} em {abs(int(delta))} ocorrências (total).")
    # aeroporto destaque
    if not top_aero.empty:
        a0 = top_aero.iloc[0]
        linhas.append(f"O aeroporto com mais atrasos foi **{a0['icao']}** com {int(a0['qtd_atrasos'])} registros >=15min.")
    # cias por ano
    if not cia_ano.empty:
        tops = cia_ano.sort_values(['ano','atrasos'], ascending=[True, False]).groupby('ano').head(1)
        for _, r in tops.iterrows():
            linhas.append(f"Em {int(r['ano'])}, a companhia com mais atrasos foi **{str(r['companhia'])}** ({int(r['atrasos'])}).")
    return '\n'.join(linhas)

insights = resumo_geral(tot_ano, top_aero, cia_ano)
print(insights or 'Sem insights automáticos (verifique colunas detectadas).')

# salva um boilerplate de storytelling em markdown
story_md = f"""# O Relógio dos Céus: como os atrasos mudaram no Brasil

**Dados**: ANAC VRA ({min(YEARS)}–{max(YEARS)}) + Airport Codes (DataHub).
**Metodologia**: atraso operacional definido como diferença entre horário realizado e previsto (partida prioritária; se indisponível, chegada). Consideramos atraso quando **>= 15 minutos**.

## O panorama
{insights or '- Sem geração automática (ajuste colunas/anos e reexecute).'}

## Gráficos
- **Dias da semana**: veja os gráficos por ano e compare padrões (ex.: picos às segundas ou sextas).
- **Períodos do dia**: manhã x tarde x noite x madrugada — qual concentra mais atrasos em cada ano?
- **Aeroportos**: top 10 no período e sua evolução ano a ano.
- Destaque para a companhia com mais atrasos em cada ano (gráficos gerados).

## Limitações
- Colunas do VRA mudam levemente ao longo do tempo. Implementamos detecção heurística (nomes contendo *prev*, *real*, *part*, *cheg*).
- Análise considera atraso >= 15 min; outras definições podem ser testadas alterando `is_atraso_15`.

*Atualizado em: {datetime.now().strftime('%Y-%m-%d %H:%M')}*
"""

with open(f'{OUTPUT_DIR}/storytelling_boilerplate.md', 'w', encoding='utf-8') as f:
    f.write(story_md)
print('Storytelling salvo em', f'{OUTPUT_DIR}/storytelling_boilerplate.md')


No período analisado (2022-2024), os atrasos aumentaram em 45816 ocorrências (total).
O aeroporto com mais atrasos foi **SBGR** com 85361 registros >=15min.
Em 2022, a companhia com mais atrasos foi **GLO** (34008).
Em 2023, a companhia com mais atrasos foi **TAM** (45548).
Em 2024, a companhia com mais atrasos foi **AZU** (49410).
Storytelling salvo em output/storytelling_boilerplate.md
